#### The purpose of this code is to pull all the necessary data to show performance of each team, using the coaches' poll point system, over time, ultimately showing which teams have been the best since 2003

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from datetime import datetime
import csv

#establish variable for DB object
CoachesPoll_DB = []

#establish variable for HTML address
espn = 'http://www.espn.com/mens-college-basketball/rankings/_/poll/2/year/'

#parse the titles of the USA Today Coaches Poll Rankings 
#table from the first Rankings page
u = espn + str('2003/week/1/seasontype/2')
r = requests.get(u)

soup = BeautifulSoup(r.content, "lxml")
tbl = soup.find("div", id = "my-teams-table")

#pull headers from USA Today Coaches Poll Rankings Table
for i in tbl.find_all("tr")[1:2]:
    head = [td.get_text().encode('utf-8') for td in i.find_all("td")]
    head = head[1:]
    head.extend(['YEAR', 'WEEK', 'SEASON'])
    CoachesPoll_DB.append(head)

In [2]:
#establish variable for current year

cur_yr = datetime.now().year

#Loop through all years from 2003 to present, all season types, and all weeks
#to pull data
for y in range (2003, cur_yr + 1):
    for q in range (1, 4):
        for x in range (1,21):
            l = espn + str(y) + '/week/' + str(x) + '/seasontype/' + str(q)
            h = requests.get(l)
            soup2 = BeautifulSoup(h.content, "lxml")
            t = soup2.find("div", id = "my-teams-table")
            for z in t.find_all("tr")[2:]:
                team_rank = [li.get_text().encode('utf-8') for li in z.find_all("li")]
                team_y = str(y)
                team_w = str(x)
                #Season Type == 2 --> Regular Season 
                if q == 2:
                    team_rank = team_rank + [team_y] + [team_w] + ['Regular Season']
                    CoachesPoll_DB.append(team_rank)
                #Season Type != 2 --> Postseason
                else:
                    team_rank = team_rank + [team_y] + [team_w] + ['Postseason']
                    CoachesPoll_DB.append(team_rank)

In [3]:
#converting list of lists to pandas dataframe
df = pd.DataFrame(CoachesPoll_DB[:],columns=CoachesPoll_DB[0])
df = df[1:]
#clean data by deleting all None from table; reindex table
df = df[pd.notnull(df['YEAR'])].reset_index(drop=True)

In [4]:
#drop all 1st place votes from team names
df['TEAM'] = (df['TEAM'].map(lambda f: f.split(' (', 1)[0]))

In [5]:
#creating Week_ID column to for future use in joining original DataFrame to a DataFrame storing unique identifying values
df['Week_ID'] = df['YEAR'] + ' ' + df['SEASON'] + ' Week ' + df['WEEK']

In [6]:
#Pulling all unique identifying values for each week of each year through present
wk_idx = pd.unique(df['Week_ID'].values.ravel())
wk_idx = pd.Series(np.arange(len(wk_idx)), wk_idx)

In [7]:
#create a second dataframe to more easily join the Week ID and Week Index to the original DataFrame
df2 = pd.DataFrame({
    'Week_ID':wk_idx.index,
    'Week_Index': wk_idx.values
    })

In [8]:
CP_df = pd.merge(df, df2, how='left', on='Week_ID') #join the two databases based on the Week_ID value
CP_df['PTS'] = CP_df['PTS'].astype(int) #convert the pts column from text to int

In [9]:
#removing unnecessary/unwanted columns
del CP_df['RECORD']
del CP_df['PREV']
del CP_df['TRENDING']
del CP_df['YEAR']
del CP_df['WEEK']
del CP_df['SEASON']

In [10]:
#create index column for easier reference in future
CP_df['Week_&_Team_Index'] = CP_df['Week_Index'].astype(str) + ' ' + CP_df['TEAM']

In [15]:
#create dataframe containing team, week #, and coaches' poll point data
k_list = list(name_dict.keys())
tempt_list = [] #temporary team list
tempi_list = [] #temporary index list
pts_list = [] #list containing coaches' poll points
wkid_list = [] #week id list

for ti in range(0, len(k_list)-1):
    for num in range(0, CP_df['Week_Index'].max()+1):
        tempt_list.append(k_list[ti])
        tempi_list.append(num)
        pts_list.append('0')
        wkid_list.append('')
df3 = pd.DataFrame({
    'TEAM': tempt_list,
    'PTS': pts_list,
    'Week_ID': wkid_list,
    'Week_Index': tempi_list,
})
df3 = df3[['TEAM', 'PTS', 'Week_ID', 'Week_Index']]
df3['Week_&_Team_Index'] = df3['Week_Index'].astype(str) + ' ' + df3['TEAM']

In [17]:
add_on = df3[~df3['Week_&_Team_Index'].isin(CP_df['Week_&_Team_Index'])]
temp_frame = [CP_df, add_on]
Total_CP_Pts_DF = pd.concat(temp_frame)
Total_CP_Pts_DF['PTS'] = Total_CP_Pts_DF['PTS'].astype(int)
Sorted_CP_df = Total_CP_Pts_DF.sort_values('Week_Index').reset_index()

In [18]:
Sorted_CP_df['Cumulative_Points'] = Sorted_CP_df['PTS'].groupby(Sorted_CP_df['TEAM']).cumsum()
del Sorted_CP_df['index']

In [20]:
df4 = pd.DataFrame({
    'Week_ID': df2['Week_ID'],
    'Week_Index': df2['Week_Index']
})

In [ ]:
#import dominant color list
team_df = pd.DataFrame.from_csv('C:\Users\E&M\Downloads\Full_Clr_DB.csv')
#reformat BGR column to separate
team_df['Color_(BGR)'] = team_df['Color_(BGR)'].str.replace(']','').str.replace('[','')
#create separate columns for B G & R values
team_df = team_df.join(pd.DataFrame(team_df['Color_(BGR)'].astype(str).str.split().tolist(),columns=['B', 'G', 'R']))

In [21]:
#output : 
df4.to_csv('week_index.csv')
Sorted_CP_df.to_csv('sortedcpdf.csv')
team_df.to_csv('team_df.csv')